In [3]:
import keras as K
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, Conv1D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
import numpy as np
import cv2
import matplotlib as plt
import os
from sklearn.model_selection import train_test_split
from datetime import datetime
from packaging import version

import tensorflow as tf

def matthews_correlation(y_true, y_pred):
    y_pred_pos = K.backend.round(K.backend.clip(y_pred, 0, 1))
    y_pred_neg = 1 - y_pred_pos

    y_pos = K.backend.round(K.backend.clip(y_true, 0, 1))
    y_neg = 1 - y_pos

    tp = K.backend.sum(y_pos * y_pred_pos)
    tn = K.backend.sum(y_neg * y_pred_neg)

    fp = K.backend.sum(y_neg * y_pred_pos)
    fn = K.backend.sum(y_pos * y_pred_neg)

    numerator = (tp * tn - fp * fn)
    denominator = K.backend.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn))

    return 1.0 - numerator / (denominator + K.backend.epsilon())

def laplacian_filter(src):
    ddepth = cv2.CV_16S
    kernel_size = 3
    src_gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)
    dst = cv2.Laplacian(src_gray, ddepth, ksize=kernel_size)
    abs_dst = cv2.convertScaleAbs(dst)
    xxx = abs_dst.shape
    abs_dst = abs_dst.reshape(xxx[0], xxx[1], 1)
    return abs_dst

def rgb2gray(rgb):
    a = np.dot(rgb[...,:3], [0.2989, 0.5870, 0.1140])
    return a

def load_data():
    laplace_data = []
    data = np.load("data.npy")
    label = np.load("label.npy")
    print(data.shape)
    print(label.shape)
    for i in range(len(data)):
        tmp = laplacian_filter(data[i])
        laplace_data.append(tmp)
    laplace_data = np.asarray(laplace_data)
    print(laplace_data.shape)
    return laplace_data, label
def model():
    # # Create model. This creates your Keras model instance, you'll use this instance to train/test the model.
    # model = Model(inputs = X_input, outputs = X, name='Recapture_Model')
    init = K.initializers.glorot_uniform(seed=1)
    rcp_model = K.models.Sequential()
    rcp_model.add(Conv2D(8, (3, 3), strides = (1, 1), name = 'conv0', padding="same", input_shape=(512,512,1)))
    rcp_model.add(BatchNormalization(axis = -1))
    rcp_model.add(Activation('relu'))
    rcp_model.add(AveragePooling2D((5, 5), padding="same", strides=(2,2)))

    #CONV2
    rcp_model.add(Conv2D(16, (3, 3), strides = (1, 1), name = 'conv1', padding="same"))
    rcp_model.add(BatchNormalization(axis = -1))
    rcp_model.add(Activation('relu'))
    rcp_model.add(AveragePooling2D((5, 5), padding="same", strides=(2,2)))

    #CONV3
    rcp_model.add(Conv2D(32, (3, 3), strides = (1, 1), name = 'conv2', padding="same"))
    rcp_model.add(BatchNormalization(axis = -1))
    rcp_model.add(Activation('relu'))
    rcp_model.add(AveragePooling2D((5, 5), padding="same", strides=(2,2)))

    #CONV4
    rcp_model.add(Conv2D(64, (3, 3), strides = (1, 1), name = 'conv3', padding="same"))
    rcp_model.add(BatchNormalization(axis = -1))
    rcp_model.add(Activation('relu'))
    rcp_model.add(AveragePooling2D((5, 5), padding="same", strides=(2,2)))

    #CONV5
    rcp_model.add(Conv2D(128, (3, 3), strides = (1, 1), name = 'conv4', padding="same"))
    rcp_model.add(BatchNormalization(axis = -1))
    rcp_model.add(Activation('relu'))
    rcp_model.add(K.layers.GlobalAveragePooling2D())
    rcp_model.add(K.layers.Dense(units=2))
    rcp_model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['acc',matthews_correlation])
    rcp_model.summary()
    return rcp_model
os.environ["CUDA_VISIBLE_DEVICES"]=""
Recapture_Model = model()
data, label = load_data()

logdir="logs/scalars/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = K.callbacks.TensorBoard(log_dir=logdir)

label = K.utils.to_categorical(label, 2).astype(np.float32)
X_train, y_train, X_test, y_test = train_test_split(data, label, test_size=0.2, random_state=42)
print("training size: {}".format(X_train.shape))
print("test size: {}".format(y_train.shape))
Recapture_Model.fit(x=X_train, y=X_test, batch_size=32, epochs=10, validation_split=0.15, callbacks=[tensorboard_callback])
Recapture_Model.save("recapture_classifier.h5")
preds = Recapture_Model.evaluate(x=y_train, y=y_test, batch_size=150)

print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv0 (Conv2D)               (None, 512, 512, 8)       80        
_________________________________________________________________
batch_normalization_6 (Batch (None, 512, 512, 8)       32        
_________________________________________________________________
activation_6 (Activation)    (None, 512, 512, 8)       0         
_________________________________________________________________
average_pooling2d_5 (Average (None, 256, 256, 8)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 256, 256, 16)      1168      
_________________________________________________________________
batch_normalization_7 (Batch (None, 256, 256, 16)      64        
_________________________________________________________________
activation_7 (Activation)    (None, 256, 256, 16)      0         
__________

W0720 10:42:16.450769 140048207583040 deprecation.py:323] From /home/liem/.local/lib/python3.6/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


training size: (2009, 512, 512, 1)
test size: (503, 512, 512, 1)


W0720 10:42:16.854662 140048207583040 deprecation_wrapper.py:119] From /home/liem/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:958: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.

W0720 10:42:16.861906 140048207583040 deprecation.py:506] From /home/liem/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:680: calling Constant.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Train on 1707 samples, validate on 302 samples


W0720 10:42:17.417466 140048207583040 deprecation_wrapper.py:119] From /home/liem/.local/lib/python3.6/site-packages/keras/callbacks.py:706: The name tf.summary.merge_all is deprecated. Please use tf.compat.v1.summary.merge_all instead.

W0720 10:42:17.417917 140048207583040 deprecation_wrapper.py:119] From /home/liem/.local/lib/python3.6/site-packages/keras/callbacks.py:709: The name tf.summary.FileWriter is deprecated. Please use tf.compat.v1.summary.FileWriter instead.



Epoch 1/10
1707/1707 [==============================] - 100s 59ms/step - loss: 1.4499 - acc: 0.7235 - matthews_correlation: 0.4710 - val_loss: 0.1180 - val_acc: 0.9603 - val_matthews_correlation: 0.1303
Epoch 2/10
1707/1707 [==============================] - 100s 59ms/step - loss: 0.1143 - acc: 0.5226 - matthews_correlation: 0.6475 - val_loss: 0.1398 - val_acc: 0.5695 - val_matthews_correlation: 0.4355
Epoch 3/10
1707/1707 [==============================] - 100s 58ms/step - loss: 0.5592 - acc: 0.6005 - matthews_correlation: 0.6778 - val_loss: 0.1684 - val_acc: 0.8377 - val_matthews_correlation: 0.2299
Epoch 4/10
1707/1707 [==============================] - 99s 58ms/step - loss: 1.4008 - acc: 0.6848 - matthews_correlation: 0.4983 - val_loss: 1.9892 - val_acc: 0.0795 - val_matthews_correlation: 1.7924
Epoch 5/10
1376/1707 [=======================>......] - ETA: 18s - loss: 0.0983 - acc: 0.5574 - matthews_correlation: 0.4602

KeyboardInterrupt: 